In [1]:
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

We follow the procedure outlined in 3.1.2.1 (Experiment 1)

Step 1: Generation of $\underline{\beta}, \underline{\gamma} \in \mathbb{R}^p$

In [2]:
from benchmark.random_k_sat import RandomKSAT

k = 8
n = 12
num_instances = 10

# Generate 100 random problem instances from CNF(12, 8, r_k)
instances = [RandomKSAT.from_poisson(n, k) for _ in range(num_instances)]

In [3]:
from k_sat.quantum_solver import QuantumSolver
from qiskit import Aer

# Statevector simulator for exact results
qi = Aer.get_backend('aer_simulator_statevector')

ps = [1] #, 4, 8, 30, 60]
cnfs = [instance.formula for instance in instances]
optimal_params = {}

for p in ps:

	# Initialise gamma_i = -0.01, beta_i = 0.01 
	initial_params = [-0.01, 0.01] * p

	qs = QuantumSolver(cnfs, layers=p, init_params=initial_params, quantum_instance=qi)

	# Find and store optimal parameters
	optimal_params[1] = qs.find_optimal_params()

KeyboardInterrupt: 